<a href="https://colab.research.google.com/github/swoo-nam/project_final_team1/blob/main/%EC%9C%A0%EC%98%88%EC%8A%AC_KoGPT_%EC%9A%94%EA%B8%B0%EC%9A%94_%EB%8B%B5%EB%B3%80%EC%83%9D%EC%84%B1_(%EC%A7%84%ED%96%89%EC%A4%91_ver2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.9 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import re
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

# DATA_PATH = "/content/drive/MyDrive/생성 AI 모델링/data/"
DATA_PATH = "/content/drive/MyDrive/멀티캠퍼스 자료/Machine Learning/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device




'cuda'

# 데이터 정제

In [4]:
yogiyo =pd.read_csv(f"{DATA_PATH}yogiyo_reviews_jsi_all.csv")


In [5]:
google_food = pd.read_csv(f"{DATA_PATH}playstore_food_reply_0904.csv")

## 정규표현식

In [6]:
train= yogiyo[['고객리뷰','사장댓글']]

rename_cols = ['review', 'reply']
train.columns = rename_cols
train.head()

,review,reply
0,사진보다 실물이 백배 맛있어요~~~~~~ 고기 양도 많아요~~~~~~ 배달시간도 딱...,"이렇게 저희 매장 찾아주셔서 감사드립니다!!\n\n맛 뿐만 아니라 양도, 서비스도 ..."
1,리뷰가 늦었네요.. 맛있게 잘 먹었습니다. 또 시킬게요!!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요😇\n\n고객님께서 맛있다고 말씀 주시...
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. \n\n저희를 찾아 주신...
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다!!\n\n저희 매장의 메뉴를 제대로 즐...
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다😂\n\n메뉴 그 자체로 행복을 드리고 ...


In [7]:
review_list = []
for review in train['review']:
    if isinstance(review, str):

        pat = re.compile("[^a-zA-Z가-힣0-9.,~?!]")
        result = pat.sub(" ", review)

        result = re.sub(r'\.{2,}', '.', result)
        result = re.sub(r'\,{2,}', ',', result)
        result = re.sub(r'\~{2,}', '~', result)
        result = re.sub(r'\!{2,}', '!', result)
        result = re.sub(r'\?{2,}', '?', result)
        result = re.sub(" +", " ", result)
        review_list.append(result)
    else:
        review_list.append("")

train['review'] = review_list
train.head()

<ipython-input-7-6374a3c305ac>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['review'] = review_list


,review,reply
0,사진보다 실물이 백배 맛있어요~ 고기 양도 많아요~ 배달시간도 딱 맞춰서 보내주셨어요~,"이렇게 저희 매장 찾아주셔서 감사드립니다!!\n\n맛 뿐만 아니라 양도, 서비스도 ..."
1,리뷰가 늦었네요. 맛있게 잘 먹었습니다. 또 시킬게요!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요😇\n\n고객님께서 맛있다고 말씀 주시...
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. \n\n저희를 찾아 주신...
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다!!\n\n저희 매장의 메뉴를 제대로 즐...
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다😂\n\n메뉴 그 자체로 행복을 드리고 ...


In [8]:
reply_list = []
for reply in train['reply']:
    if isinstance(reply, str):

        pat = re.compile("[^a-zA-Z가-힣0-9.,~?!]")
        result = pat.sub(" ", reply)

        result = re.sub(r'\.{2,}', '.', result)
        result = re.sub(r'\,{2,}', ',', result)
        result = re.sub(r'\~{2,}', '~', result)
        result = re.sub(r'\!{2,}', '!', result)
        result = re.sub(r'\?{2,}', '?', result)
        result = re.sub(" +", " ", result)
        reply_list.append(result)
    else:
        reply_list.append("")

train['reply'] = reply_list
train.head()

<ipython-input-8-2764c488e75c>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['reply'] = reply_list


,review,reply
0,사진보다 실물이 백배 맛있어요~ 고기 양도 많아요~ 배달시간도 딱 맞춰서 보내주셨어요~,"이렇게 저희 매장 찾아주셔서 감사드립니다! 맛 뿐만 아니라 양도, 서비스도 모두 최..."
1,리뷰가 늦었네요. 맛있게 잘 먹었습니다. 또 시킬게요!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요 고객님께서 맛있다고 말씀 주시니 피로...
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다! 저희 매장의 메뉴를 제대로 즐기신 거...
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다 메뉴 그 자체로 행복을 드리고 싶었는데...


## 1) 고객id 처리
## - ~님 -> 고객님

In [9]:
train['reply'] = train['reply'].str.replace(r'\w+\s*님', '고객님', regex=True)


<ipython-input-9-ef52f5ba6171>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['reply'] = train['reply'].str.replace(r'\w+\s*님', '고객님', regex=True)


In [13]:

for review in train['reply']:
  if '고객님' in review:
      print(review)
      break

이렇게 저희 매장 찾아주셔서 감사드립니다! 맛 뿐만 아니라 양도, 서비스도 모두 최상의 품질을 드리려고 노력하고 있습니다 모든 직원들이 힘내서 고객님의 마음을 잡기 위해 계속 노력하겠습니다 항상 건강 챙기시고, 오늘도 즐거운 하루 보내시길 바라겠습니다 


## 2) 가게명 처리
- reply에 가게명이 들어가있는 경우에 store 칼럼에 가게명 기재, 모델이 함께 학습할 수 있도록 처리
- '저희 가게'로 처리?



In [14]:
ygy_store_lst = ["1988응답하라추억의옛날도시락-방이점", "워커스커피 로스터스", "떡깨비-가락점", "당치땡-헬리오시티점", "우시장국밥-석촌점",  "고돼지-송파점",  "마라의신마라탕&마라샹궈",  "춘리마라탕-송파점",  "손수경의육회담은연어-가락점",  "무국적식탁-송파점",  "청년치킨-삼전점",  "송파만다린",  "배달의쌀국수-송파점",  "닭강점-헬리오시티점",  "토핑폭탄김치찜&김치찌개-송파점",  "조형훈족발보쌈",  "여왕의브런치",  "모어댄버터",  "플러스82-송파점",  "더티베지",  "잭아저씨족발보쌈-본점",  "육회&연어-미아본점",  "갈비민족-본점",  "구구족-성신여대역점",  "피자보이시나-대학로점",  "왕빈자삼파전-미아본점",  "행복한찜닭-성북점",  "스시사소우",  "저팔계&중국&마라탕&덮밥&요리&전문점",  "명작파스타-성북점",  "올데이파스타-동대문구점",  "대한냉면-성북점",  "인생제육",  "고기듬뿍미트박스-성북점",  "피자스웨그-성북점",  "청춘식당묵은지김치찜",  "삽교원조두리곱창",  "뜸들이다-북서울꿈의숲점",  "더팔당매운오징어&닭발 김치찜",  "우리집반찬도시락-혜화막내딸지점",  "수피자",  "마라쿡",  "김포불닭발",  "헬로팬닭갈비&마약볶음밥",  "대찌 대파생고기김치찌개&정육왕국물두루치기",  "구래상회",  "승도리네곱도리탕",  "자성당",  "수락",  "나인곱창",  "어시장",  "돈까스회관",  "호랑이초밥",  "땡초곱창막창",  "피자이탈리",  "텐텐마라탕",  "과일에반하다",  "삼다수",  "병천토속순대",  "최고남제육명가",  "짱닭치킨",  "걸작떡볶이",  "직화삼겹직구삼",  "짜글이가돼었소",  "부대찌개자신있는집",  "토핑폭탄김치찜&김치찌개-부천점",  "1989마라탕-부천점",  "미친피자-부평점",  "천년닭강정-부천부평점",  "수상한삼겹살-부천점",  "수제죽전문점-본가진죽-본점",  "오늘은 분식",  "이끌림마라탕",  "탐나는피자-부평점",  "고기혁명-부천점",  "라화방마라탕-신중동점",  "전금례닭볶음탕-부천점",  "큰아들백순대",  "공주닭발-부천시청점",  "신자매김치찜&김치찌개",  "짱닭치킨-도촌점",  "이태리면가게-야탑점",  "폴트버거-판교점",  "랜돌프뉴욕페페로니-야탑점",  "홍싸롱-수제돈까스&파스타",  "쇼부덮밥",  "카산도-정자본점",  "부성초밥-미금본점",  "닭장수섭삼계탕&전기구이통닭",  "깨돌이김밥-미금점",  "울진죽변항",  "GTS버거-분당정자점",  "코브라독스-분당미금역점",  "마라하오-죽전점",  "땅끝수산",  "초밥대통령-용인수지점",  "올데이케밥&샐러드",  "야키토리 카마쿠라",  "비오키친-서현본점",  "부산오빠1인국밥-분당점",  "알촌",  "밈피자",  "고씨네카레",  "홍스족발",  "뜸들이다",  "감성커피",  "짚신스시",  "떡형",  "39도시락",  "피제이피자",  "황궁쟁반짜장",  "OK반점",  "모모타코야키&카페",  "꼬알라파이",  "진지덮밥",  "피자프라텔로",  "한식세끼1인김치찜&김치찌개",  "윤희횟집",  "시민보쌈족발&감자탕",  "카페봄봄",  "킹프레소빅와플",  "최가네한쌈",  "창타이누들",  "보돌미역",  "홍대개미"  ]


In [17]:
store_name = []

for store in ygy_store_lst:
    pat = re.split(r'[- &]', store)
    store_name.append(pat[0])

len(store_name)

125

In [18]:
store_name[:20]

['1988응답하라추억의옛날도시락',
 '워커스커피',
 '떡깨비',
 '당치땡',
 '우시장국밥',
 '고돼지',
 '마라의신마라탕',
 '춘리마라탕',
 '손수경의육회담은연어',
 '무국적식탁',
 '청년치킨',
 '송파만다린',
 '배달의쌀국수',
 '닭강점',
 '토핑폭탄김치찜',
 '조형훈족발보쌈',
 '여왕의브런치',
 '모어댄버터',
 '플러스82',
 '더티베지']

In [19]:
# train['store'] = ""  # full store name을 칼럼에 담기
# for name in store_name:
#     train[name] = train['reply'].apply(lambda x: name if name in x else '') # 우선 각 가게명 컬럼 생성(그렇게해야 덮어쓰기가 안됨)
#     train['store'] = train.apply(lambda row: row['store'] + ',' + row[name] if row[name] else row['store'], axis=1) # 하나로 합치기(맨 앞에 콤마가 생김)

# train['store'] = train['store'].apply(lambda x: x[1:] if x.startswith(',') else x) # 맨 앞에 콤마 제거
# train = train.drop(columns=store_name) # 각 가게명 컬럼 삭제

# train.head()

- 1차 : store 이름이 풀로 들어간 경우 <store>로 대체

In [ ]:
def replace_store_names1(text):
    for name1 in store_name:
        if name1 in text:
            # 가게 이름의 일부가 텍스트에 포함되어 있다면 대체
            pattern = re.compile(re.escape(name1), re.IGNORECASE)
            text = pattern.sub("<store>", text)
    return text

In [ ]:
replace_store_names1("1988응답하라추억의옛날도시락에서 먹은 음식은 정말 맛있었습니다.")

'<store>에서 먹은 음식은 정말 맛있었습니다.'

In [ ]:
replace_store_names1("킹프레소에서 먹은 음식은 정말 맛있었습니다.") # 안바뀜

'킹프레소에서 먹은 음식은 정말 맛있었습니다.'

- 2차 : store 이름이 부분적으로 들어간 경우 store로 대체

In [ ]:
def replace_store_names2(text):
    for name1 in tqdm(store_name, desc="Processing"):
        for i in range(len(name1), 1, -1):
            part = name1[:i]  # 가게 이름에서 처음부터 i 글자까지의 부분
            if part in text:
                # 가게 이름의 일부가 텍스트에 포함되어 있다면 대체
                pattern = re.compile(re.escape(part), re.IGNORECASE)
                text = pattern.sub("<store>", text)
    return text


In [ ]:
replace_store_names2("텐텐에서 먹은 음식은 정말 맛있었습니다.") # 텐텐마라탕

Processing:   0%|          | 0/125 [00:00<?, ?it/s]

'<store>에서 먹은 음식은 정말 맛있었습니다.'

In [ ]:
replace_store_names2("킹프레소에서 먹은 음식은 정말 맛있었습니다.") # 킹프레소

Processing:   0%|          | 0/125 [00:00<?, ?it/s]

'<store>에서 먹은 음식은 정말 맛있었습니다.'

- 3차 : 일반명사 등은 제외하고 대체해야 함

In [ ]:
!pip install kiwipiepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 44.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kiwipiepy-model: filename=kiwipiepy_model-0.16.0-py3-none-any.whl size=30813255 sha256=77dfbd6a9a154265e143c9b0974b91a935a585d88b71366c6d6a201dd09af3c0
  Stored in directory: /root/.cache/pip/wheels/6b/03/2b/ac2c97cc65ebd9df3516f4b900adc2f0a744df8d1375b2e2ef
Successfully built kiwipiepy-model


In [ ]:
# # short_name 에서 가게명 이외 일상용어로 사용되는 키워드는 제거
# drop_name = ['고기','OK','감성','걸작','공주','과일','김포','대한','땡초','명작','미친','배달','병천','부대','부산','비오','삽교',
#              '손수','송파','쇼부','수상','수제','시민','오늘', '오늘은', '우리', '울진', '이끌', '이끌림', '인생', '직화', '진지',
#              '천년', '청년', '청춘', '최고', '큰아', '큰아들', '탐나', '토핑', '행복', '행복한', '헬로', '홍대',
#              '육회','수락','호랑','갈비','무국','닭장','한식','마라','스시','여왕','돈까','초밥','카페','피자','부대찌','이태리','플러','플러스']
# # drop_name = ['과일','오늘', '오늘은','우리', '이끌', '이끌림', '인생','진지','최고''토핑', '행복', '행복한', '한식', '감성','걸작']

In [ ]:
import kiwipiepy
kiwi = kiwipiepy.Kiwi()

In [ ]:
def remove_adjectives_nouns(text):
    result = []
    tokens = kiwi.extract_words(text, min_cnt=1, min_score=0.25)
    for token in tokens:
        word = token[0].form  # 단어만 추출하여 사용
        pos = token[1]
        # 형용사나 명사가 아닌 경우에만 결과에 추가
        if pos not in ('VA', 'NNG'):
            result.append(word)
    return ' '.join(result)


In [ ]:
tokens = kiwi.analyze(store_name)

remove_words = []

for token in tokens :
  word = token[0][0][0].form
  pos = token[0][0][0].tag
  if pos in ('NNG') and len(word) ==2 :
    print(word)
    remove_words.append(word)

# 2글자 일반명사 단어들은 제외

춘리
청년
배달
토핑
조형
여왕
모어
육회
갈비
피자
행복
명작
인생
고기
청춘
헬로
대찌
구래
자성
수락
나인
땡초
피자
과일
삼다
최고
걸작
직화
토핑
천년
수제
오늘
고기
공주
홍싸
쇼부
부성
초밥
감성
짚신
진지
한식
시민
카페
보도


In [ ]:
tokens = kiwi.analyze(store_name)

for token in tokens :
  word = token[0][0][0].form
  pos = token[0][0][0].tag
  if pos in ('NNG') and len(word) ==3 :
    print(word)

# 3글자 일반명사 단어들은 가게명으로 포함해도될듯

떡깨비
우시장
플러스
더티베
돈까스
호랑이
라화방
큰아들
피제이


In [ ]:
tokens = kiwi.analyze(store_name)

for token in tokens :
  word = token[0][0][0].form
  pos = token[0][0][0].tag
  if pos in ('NNG') and len(word) ==4 :
    print(word)

# 4글자 일반명사 단어들은 가게명으로 포함해도될듯

부대찌개
폴트버거
창타이누


In [ ]:
store_name[:5]

['1988응답하라추억의옛날도시락', '워커스커피', '떡깨비', '당치땡', '우시장국밥']

- 최종 합수

In [ ]:
# def replace_store_names(text):
#     for name1 in store_name:
#         # 먼저 가게 이름 전체가 텍스트에 포함되어 있는지 확인
#         if name1 in text:
#             # 가게 이름 전체가 텍스트에 포함되어 있다면 대체
#             pattern = re.compile(re.escape(name1), re.IGNORECASE)
#             text = pattern.sub("<store>", text)
#         else:
#             # 가게 이름의 일부가 텍스트에 포함되어 있는 경우를 확인
#             for i in range(len(name1), 1, -1):
#                 part = name1[:i]  # 가게 이름에서 처음부터 i 글자까지의 부분
#                 if part in text:
#                     # 가게 이름의 일부가 텍스트에 포함되어 있다면 대체
#                     pattern = re.compile(re.escape(part), re.IGNORECASE)
#                     text = pattern.sub("<store>", text)
#                     break  # 가게 이름 일부를 대체했으면 루프 종료
#     return text


In [ ]:
def replace_store_names(text):
    for name1 in store_name:
        # 먼저 가게 이름 전체가 텍스트에 포함되어 있는지 확인
        if name1 in text:
            # 가게 이름 전체가 텍스트에 포함되어 있다면 대체
            pattern = re.compile(re.escape(name1), re.IGNORECASE)
            text = pattern.sub("<store>", text)
        else:
            # 가게 이름의 일부가 텍스트에 포함되어 있는 경우를 확인
            for i in range(len(name1), 1, -1):
                part = name1[:i]  # 가게 이름에서 처음부터 i 글자까지의 부분
                if part in text:
                    # 가게 이름의 일부가 텍스트에 포함되어 있고, 해당 부분이 removelist에 없는 경우에만 대체
                    if part not in remove_words:
                        pattern = re.compile(re.escape(part), re.IGNORECASE)
                        text = pattern.sub("<store>", text)
                    break  # 가게 이름 일부를 대체했으면 루프 종료
    return text


In [ ]:
remove_words

['춘리',
 '청년',
 '배달',
 '토핑',
 '조형',
 '여왕',
 '모어',
 '육회',
 '갈비',
 '피자',
 '행복',
 '명작',
 '인생',
 '고기',
 '청춘',
 '헬로',
 '대찌',
 '구래',
 '자성',
 '수락',
 '나인',
 '땡초',
 '피자',
 '과일',
 '삼다',
 '최고',
 '걸작',
 '직화',
 '토핑',
 '천년',
 '수제',
 '오늘',
 '고기',
 '공주',
 '홍싸',
 '쇼부',
 '부성',
 '초밥',
 '감성',
 '짚신',
 '진지',
 '한식',
 '시민',
 '카페',
 '보도']

In [ ]:
korean_particles = ['은', '는', '이', '가', '을', '를', '의', '한', '로', '으로', '까지', '이며', '며', '와', '과']

#위 일반명사에, 조사를 합친 경우도 제외함

In [ ]:
remove_withparticles = []

for word in remove_words :
  for particle in korean_particles:
    new_word = word + particle
    remove_withparticles.append(new_word)

In [ ]:
def replace_store_names(text):
    for name1 in store_name:
        # 먼저 가게 이름 전체가 텍스트에 포함되어 있는지 확인
        if name1 in text:
            # 가게 이름 전체가 텍스트에 포함되어 있다면 대체
            pattern = re.compile(re.escape(name1), re.IGNORECASE)
            text = pattern.sub("<store>", text)
        else:
            # 가게 이름의 일부가 텍스트에 포함되어 있는 경우를 확인
            for i in range(len(name1), 1, -1):
                part = name1[:i]  # 가게 이름에서 처음부터 i 글자까지의 부분
                if part in text:
                    # 가게 이름의 일부가 텍스트에 포함되어 있고, 해당 부분이 remove_words에 없으며,
                    # 해당 부분 뒤에 조사가 붙지 않은 경우에만 대체
                    if part not in remove_words + remove_withparticles :
                        pattern = re.compile(re.escape(part), re.IGNORECASE)
                        text = pattern.sub("<store>", text)
                    break  # 가게 이름 일부를 대체했으면 루프 종료
    return text


In [ ]:
replace_store_names('행복한 라화방마라탕에서 맛있게 식사했어요') #행복 부분이 대체안됨

'행복한 <store><store>탕에서 맛있게 식사했어요'

In [ ]:
# train['reply']의 답변들에 함수 적용하기 / 18000개 7분 정도 걸림

for i, review in tqdm(enumerate(train['reply']), desc="Processing", ncols=100):
    train['reply'][i] = replace_store_names(review)

Processing: 0it [00:00, ?it/s]

In [ ]:
changed_rows = train[train['reply'].str.contains("<store>")] #총 9446개
changed_rows

,review,reply,store
16,저저번달에 맛있게 먹어서 또 시켰는데 고기 양이 엄청 줄었네요 .된장찌개도 엄청 짜...,저희 <store>의 메뉴를 주문해 주셔서 감사합니다. 양이 줄었다고 느끼셨군요. ...,1988응답하라추억의옛날도시락
23,20분 이내 배달 알림 주시고는 한시간이 지나서야 받았습니다. 배달이 늦는 점에 대...,안녕하세요 <store> 입니다. 남겨 주신 리뷰 잘 읽어보았습니다. 먼저 매장 상...,1988응답하라추억의옛날도시락
26,정말 맛있어요. 또 시켜 먹고 싶네요.,오늘도 저희 매장에 귀한 발걸음을 해주셔서 영광입니다 맛에 <store> 칭찬은 정...,
28,집찌개 생각나서 시켰는데 좋았어요. 집보다는 학교 다닐 때 먹었던 김치찌개 맛이긴 ...,마음이 따뜻해지는 리뷰를 안고 와주셨네요 배달로 드시는 만큼 제가 더 해드릴 수 있...,
29,양도 많고 진짜 맛있어요! 시원한 묵사발로 더위도 가라앉히고 밥이랑 고기 양도 많아...,"저희 매장을 선택해주시고, 리뷰 남겨주셔서 감사합니다! 메뉴를 드시다가 양이 모자라...",
...,...,...,...
18537,비오는 날인데도 배달 잘 와서 감사합니다,비가 와도 <store>! 신메뉴3종 출시 되었으니 앞으로도 많은 애용 부탁드립니당...,홍대개미
18544,맛있게 보이시죠? 맛있어요 드셔보세요,매번 감사드립니다! 오늘도 역시나 리뷰 잘 써주셔서 감사드립니다! 월요일마다 <st...,
18569,마시써욥 소스양념 같은거 많이 넣어달라고 부탁했는데 더 넣어주신게 느껴졌어용 굿굿 ...,사진은 <store>비빔밥인가요? 신메뉴3종 출시 되었으니 앞으로도 많은 애용 부탁...,
18662,예상시간 보다 빠르게 보내주셔서 넘 맛나게 잘 먹었습니다,맛있게 드셔주셔서 감사합니다 요즘 비가 자주와서 혹시몰라 길게 잡아두지만 최<sto...,


In [ ]:
# train['review']의 리뷰들에 함수 적용하기 / 7분 정도 걸림

for i, review in tqdm(enumerate(train['review']), desc="Processing", ncols=100):
    train['review'][i] = replace_store_names(review)

Processing: 0it [00:00, ?it/s]

In [ ]:
changed_rows = train[train['review'].str.contains("<store>")] #총 25개
changed_rows

,review,reply,store
23,20분 이내 배달 알림 주시고는 한시간이 지나서야 받았습니다. 배달이 늦는 점에 <...,안녕하세요 <store> 입니다. 남겨 주신 리뷰 잘 읽어보았습니다. 먼저 매장 상...,1988응답하라추억의옛날도시락
37,삼겹도시락 맛났어요~ 이 가격에 <store>은 인정~ 근데 서비스가 아니라 추가 ...,오늘 저희 매장에 발걸음을 해주셔서 영광입니다 메뉴 시키실 때마다 항상 많이 고민하...,
196,글레이즈드 도넛은 레몬의 상큼함과 달달함. 커피는 정말 고소하니 좋았고 와플은 <s...,안녕하세요 주문해주셔서 감사드립니다 피로가 풀리는 커피와 함께 하는 디저트 조합! ...,
212,실수로 와플을 누락하고 배달 받았는데 사장님이 바로 다시 가져다 주신다고 하고 서비...,안녕하세요! 주문해주셔서 감사드립니다 달콤한 디저트 커피와 함께 즐겁게 드셨기를 바...,
230,라떼는 <store>가 제일 맛있는거 같아요 요즘은 우유 안먹으려고 귀리우유로 주문...,안녕하세요 소중한 리뷰는 감동입니다! 소이라떼에 들어가는 귀리우유 좋은거 사용하고 ...,
...,...,...,...
18618,제가 먹어본 <store> 중에 양 제일 많았어요! 배달도 빨리 오고 양파 조금 더...,극찬 너무 감사드립니다 맛있게 드셔주셔서 감사합니당 리뷰 감사합니다! 다음에 또 ...,
18636,스테이크 굽기 좋아서 너무 맛있게 먹었습니다. 고기 추가가 없어 아쉬웠지만. 덕분에...,고객님 요청으로 드디어 요기요에도 고기추가 추가 했습니다! 배.민에 이어 요기요 까...,
18641,믿고먹는 <store> 짱짱맛있어요 넘 맛있어서 좋아요,맛있게 드셔주셔서 감사합니당 리뷰 감사합니다! 다음에 또 뵙겠습니다,
18678,<store> 넘 마싯어요 <store> 놀러가서 첫끼가 <store>였는대 참 순...,헉 <store>에서 처음.! 감사합니당,


In [ ]:
# len(set(train['reply'].tolist())) # 우선은 중복제거 안함. 어떤 유형의 리뷰에 대해 동일답변하는 하는지 체크 필요

In [ ]:
store_mask = []

for review in train['reply']:
  if '<store>' in review:
      print(review)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
안녕하세요 곱도리탕 원조 <store> <store>풍무점입니다~ 귀한 시간내어 남겨주신 별점과 리뷰의 말씀 너무나 감사합니다 저희 승도리네<store>풍무점 언제나 고객님의 소중한 한끼의 식사를 책임진다는 마음으로 정성껏 대접하여 드리겠습니다. 든든한 한끼의 식사가 필요하실땐, <store><store>풍무점 재주문도 부탁드려요~ 그럼 오늘도 행복하고 건강한날되세요~ 다시한번 주문하여주심에 깊은 감사드려요! 
안녕하세요 곱도리탕 원조 <store> <store>풍무점입니다~ 귀한 시간내어 남겨주신 별점과 리뷰의 말씀 너무나 감사합니다 맛있고 행복한 식사시간이 되신듯하여,저희의 마음이 뿌듯하고 행복합니다~ 든든한한끼의 식사가 필요하신날에, <store><store>풍무점 재주문도 부탁드리며, 다시 뵙게 되는 그날에도 넉넉한 양과 정성으로 대접하여 드리겠습니다 그럼 오늘도 행복하고 건강한날되세요~ 다시한번 주문하여주심에 깊은 감사드려요! 
원조 곱도리탕 <store> <store>풍무점 입니다. 맛있게 드셨다니 다행이고 고객님의 소중한 시간 저희 곱도리탕을 제공할 수 있게 되어 영광입니다. 저희 곱도리탕은 해장,식사,술안주 동시에 3가지를 즐기실 수 있습니다 언제나 고객님의 소중한 시간 푸짐한 한 끼를 대접해 드릴 수 있도록 노력하고 기대이상의 맛과 친절,위생에 최선을 다하는 <store>가 되겠습니다. 소중한 시간 내어 소중한 리뷰 정말 감사합니다! 
원조 곱도리탕 <store> <store>풍무점 입니다. 맛있게 드셨다니 다행이고 고객님의 소중한 시간 저희 곱도리탕을 제공할 수 있게 되어 영광입니다. 항상 친절은 기본! 고객님의 소중한 시간 언제나 만족을 드릴 수 있도록 최선을 다하겠습니다. 소중한 시간 내어 소중한 리뷰 정말 감사합니다! 고객님의 소중한 시간 푸짐한 한 끼를 대접해 드릴 수 있도록 노력하고 기대이상의 맛과 친절,위생에 최선을 다하는 <store>가 되겠습니다. 다시한번 주문주셔

In [ ]:
# store_mask = []   # 확인용

# for review in train['reply']:
#   if '<store>' not in review:
#       print(review)

In [ ]:
# short_store_name = []
# for name in store_name:
#     for length in [2, 3, 4,5,6]:
#         cut = name[:length]
#         short_store_name.append(cut)
#     # if name[0].isdigit(): # 숫자로 시작하면 텍스트가 나오는 부분까지 추출
#     #     cut = name[:6]
#     #     short_name.append(cut)

# short_store_name = [name for name in short_store_name if not name.isdigit()] # 숫자로만 되어 있는 키워드 삭제
# short_store_name = list(set(short_store_name))
# print(len(short_store_name), short_store_name)



- 형용사, 부사, 등등인 경우는 제외하기 / 등등의 방법 필요함




In [ ]:
# # short_name 에서 가게명 이외 일상용어로 사용되는 키워드는 제거
# drop_name = ['고기','OK','감성','걸작','공주','과일','김포','대한','땡초','명작','미친','배달','병천','부대','부산','비오','삽교',
#              '손수','송파','쇼부','수상','수제','시민','오늘', '오늘은', '우리', '울진', '이끌', '이끌림', '인생', '직화', '진지',
#              '천년', '청년', '청춘', '최고', '큰아', '큰아들', '탐나', '토핑', '행복', '행복한', '헬로', '홍대',
#              '육회','수락','호랑','갈비','무국','닭장','한식','마라','스시','여왕','돈까','초밥','카페','피자','부대찌','이태리','플러','플러스']
# # drop_name = ['과일','오늘', '오늘은','우리', '이끌', '이끌림', '인생','진지','최고''토핑', '행복', '행복한', '한식', '감성','걸작']

# # 명사인 경우에만 남기기 형용사, 부사 등이
# # 청년 치킨

# short_store_name = [col for col in short_store_name if col not in drop_name]
# print(len(short_store_name),short_store_name)

In [ ]:
# store_name = store_name + short_store_name  # 두 그룹은 set처리 안함. store name(full)이 우선적으로 처리되어야 하기 때문

In [ ]:
len(store_name)

125

In [ ]:
# def replace_store_names(text, store_name):
#     for store in store_name:
#         text = text.replace(store, "<store>")
#     return text

# for store in store_name:
#     for i, review in enumerate(train['reply']):
#         if store in review:
#             train['reply'][i] = replace_store_names(review, [store])

In [ ]:
# mask = train['store'] != ''
# train[mask]    #약 1/3 6330개 리뷰답변에 가게명 확인 / 글자 2~7 추가 -> 11379 리뷰

- store 부분을 <store> 로 처리하기

## 3) 지점명 처리
- 전국일반음식점 표준 데이터 : https://www.data.go.kr/data/15096283/standard.do
- 행정구역 리스트 -> 리스트 내에 있는 단어인 경우 확인해서 region 칼럼에 추가 or 삭제
- 지점명의 경우 보통 가게명과 함께 등장하기 때문에, 빈칸으로 삭제해도 무방

- ~호점 삭제

In [ ]:
# 정규 표현식을 사용하여 "~호점" 패턴을 찾아서 삭제하는 함수
def remove_hojum(text):
    return re.sub(r'\b\w+호점\b', '', text)

# train 데이터프레임의 'reply' 열에 적용하여 "~호점"을 삭제
train['reply'] = train['reply'].apply(remove_hojum)
train['review'] = train['review'].apply(remove_hojum)

- 지점명을 특정 태그로 변경

In [ ]:
# restaurant_seoul = pd.read_csv(f"{DATA_PATH}소상공인시장진흥공단_상가(상권)정보_서울_202212.csv")

In [ ]:
restaurant_list = pd.read_csv(f"{DATA_PATH}fulldata_07_24_04_P_일반음식점.csv",encoding='cp949')

<ipython-input-54-afda4487c689>:1: DtypeWarning: Columns (11,27,39,41,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  restaurant_list = pd.read_csv(f"{DATA_PATH}fulldata_07_24_04_P_일반음식점.csv",encoding='cp949')


In [ ]:
restaurant_list['소재지전체주소']

0                   서울특별시 동대문구 장안동 382-1
1               서울특별시 마포구 망원동 386-2번지 지층
2                 서울특별시 마포구 서교동 403-14번지
3              서울특별시 마포구 서교동 363-5번지 B1동
4          경기도 안양시 동안구 호계동 930-30번지 지상1층
                       ...              
2086169             광주광역시 서구 용두동 306번지 1
2086170              광주광역시 남구 양림동 374 2층
2086171     광주광역시 남구 월산동 1048-182번지 (1층)
2086172        광주광역시 서구 화정동 332-9번지 (1층)
2086173          광주광역시 서구 화정동 70-61 (1층)
Name: 소재지전체주소, Length: 2086174, dtype: object

In [ ]:
# 가게명에서 지점명 추출하기

store_etc_name = []

for store in ygy_store_lst:
    pat = re.split(r'[- &]', store)
    for name in pat[1:]:
        if name != '':
            store_etc_name.append(name)

ygy_region_name = []

for region in store_etc_name:
  if region[-1] =='점':
    ygy_region_name.append(region)

In [ ]:
address_parts=[]

for address in restaurant_list['소재지전체주소'].astype(str):
    if address.startswith('서울'or '부산'):
        parts = address.split(' ')
        for _ in range(0, min(4, len(parts))):  # 최대 5개 또는 주소 부분 개수만큼 반복
          part = parts[_]
          address_parts.append(part)

address_parts = set(address_parts)
address_parts = list(address_parts)

address_parts = [name for name in address_parts if all(char.isalpha() for char in name)] #알파벳으로만 (한글) 구성된 경우남김. 숫자, 특수문자 지움
address_parts


['',
 '내곡동',
 '불광동',
 '광장동',
 '보광동',
 '마포구',
 '무학동',
 '홍제동',
 '행촌동',
 '계동',
 '당산동',
 '여의도동',
 '누하동',
 '관수동',
 '송정동',
 '이태원동',
 '남학동',
 '대조동',
 '자양동',
 '상왕십리동',
 '초동',
 '내자동',
 '사직동',
 '구기동',
 '상봉동',
 '가리봉동',
 '묵정동',
 '창신동',
 '양화동',
 '강동구',
 '도화동',
 '화양동',
 '고덕시영',
 '봉익동',
 '구수동',
 '성북구',
 '동자동',
 '쌍문동',
 '잠실동',
 '도곡동',
 '신월동',
 '미아동',
 '옥수동',
 '행당동',
 '도원동',
 '오쇠동',
 '쌍림동',
 '합동',
 '마곡동',
 '봉원동',
 '와룡동',
 '주교동',
 '상일동',
 '충신동',
 '당주동',
 '북창동',
 '시흥동',
 '방화동',
 '방산동',
 '신공덕동',
 '서린동',
 '강일동',
 '공릉동',
 '월계동',
 '오장동',
 '응봉동',
 '동대문구',
 '내발산동',
 '정릉동',
 '녹번동',
 '소공동',
 '원남동',
 '현저동',
 '이촌동',
 '부암동',
 '홍익동',
 '압구정동',
 '길동',
 '문배동',
 '인사동',
 '과해동',
 '평동',
 '잠원동',
 '통인동',
 '혜화동',
 '중학동',
 '연지동',
 '강북구',
 '현석동',
 '삼성동',
 '갈현동',
 '남현동',
 '명일동',
 '천왕동',
 '동빙고동',
 '영천동',
 '동작동',
 '무악동',
 '오류동',
 '장위동',
 '구의동',
 '화곡동',
 '가산동',
 '신대방동',
 '석촌동',
 '북가좌동',
 '수송동',
 '필운동',
 '온수동',
 '소격동',
 '구로구',
 '강서구',
 '연남동',
 '신원동',
 '역삼동',
 '휘경동',
 '냉천동',
 '성동구',
 '망우동',
 '송파동',
 '하중동',
 '재동',
 '노고산동',
 '송현동',
 '모

In [ ]:
address_parts2 = []

for address in address_parts :
  if len(address) < 3  :
    address = address + '점'
    address_parts2.append(address)
  else :
    address = address[:-1] + '점'
    address_parts2.append(address)

In [ ]:
address_parts2.remove('점')
address_parts2

['내곡점',
 '불광점',
 '광장점',
 '보광점',
 '마포점',
 '무학점',
 '홍제점',
 '행촌점',
 '계동점',
 '당산점',
 '여의도점',
 '누하점',
 '관수점',
 '송정점',
 '이태원점',
 '남학점',
 '대조점',
 '자양점',
 '상왕십리점',
 '초동점',
 '내자점',
 '사직점',
 '구기점',
 '상봉점',
 '가리봉점',
 '묵정점',
 '창신점',
 '양화점',
 '강동점',
 '도화점',
 '화양점',
 '고덕시점',
 '봉익점',
 '구수점',
 '성북점',
 '동자점',
 '쌍문점',
 '잠실점',
 '도곡점',
 '신월점',
 '미아점',
 '옥수점',
 '행당점',
 '도원점',
 '오쇠점',
 '쌍림점',
 '합동점',
 '마곡점',
 '봉원점',
 '와룡점',
 '주교점',
 '상일점',
 '충신점',
 '당주점',
 '북창점',
 '시흥점',
 '방화점',
 '방산점',
 '신공덕점',
 '서린점',
 '강일점',
 '공릉점',
 '월계점',
 '오장점',
 '응봉점',
 '동대문점',
 '내발산점',
 '정릉점',
 '녹번점',
 '소공점',
 '원남점',
 '현저점',
 '이촌점',
 '부암점',
 '홍익점',
 '압구정점',
 '길동점',
 '문배점',
 '인사점',
 '과해점',
 '평동점',
 '잠원점',
 '통인점',
 '혜화점',
 '중학점',
 '연지점',
 '강북점',
 '현석점',
 '삼성점',
 '갈현점',
 '남현점',
 '명일점',
 '천왕점',
 '동빙고점',
 '영천점',
 '동작점',
 '무악점',
 '오류점',
 '장위점',
 '구의점',
 '화곡점',
 '가산점',
 '신대방점',
 '석촌점',
 '북가좌점',
 '수송점',
 '필운점',
 '온수점',
 '소격점',
 '구로점',
 '강서점',
 '연남점',
 '신원점',
 '역삼점',
 '휘경점',
 '냉천점',
 '성동점',
 '망우점',
 '송파점',
 '하중점',
 '재동점',
 '노고산점',
 '송현점',
 '

In [ ]:
len(address_parts2)

359

In [ ]:
region_name = ygy_region_name + address_parts2

In [ ]:
region_name

['방이점',
 '가락점',
 '헬리오시티점',
 '석촌점',
 '송파점',
 '송파점',
 '가락점',
 '송파점',
 '삼전점',
 '송파점',
 '헬리오시티점',
 '송파점',
 '송파점',
 '본점',
 '미아본점',
 '본점',
 '성신여대역점',
 '대학로점',
 '미아본점',
 '성북점',
 '전문점',
 '성북점',
 '동대문구점',
 '성북점',
 '성북점',
 '성북점',
 '북서울꿈의숲점',
 '혜화막내딸지점',
 '부천점',
 '부천점',
 '부평점',
 '부천부평점',
 '부천점',
 '본점',
 '부평점',
 '부천점',
 '신중동점',
 '부천점',
 '부천시청점',
 '도촌점',
 '야탑점',
 '판교점',
 '야탑점',
 '정자본점',
 '미금본점',
 '미금점',
 '분당정자점',
 '분당미금역점',
 '죽전점',
 '용인수지점',
 '서현본점',
 '분당점',
 '내곡점',
 '불광점',
 '광장점',
 '보광점',
 '마포점',
 '무학점',
 '홍제점',
 '행촌점',
 '계동점',
 '당산점',
 '여의도점',
 '누하점',
 '관수점',
 '송정점',
 '이태원점',
 '남학점',
 '대조점',
 '자양점',
 '상왕십리점',
 '초동점',
 '내자점',
 '사직점',
 '구기점',
 '상봉점',
 '가리봉점',
 '묵정점',
 '창신점',
 '양화점',
 '강동점',
 '도화점',
 '화양점',
 '고덕시점',
 '봉익점',
 '구수점',
 '성북점',
 '동자점',
 '쌍문점',
 '잠실점',
 '도곡점',
 '신월점',
 '미아점',
 '옥수점',
 '행당점',
 '도원점',
 '오쇠점',
 '쌍림점',
 '합동점',
 '마곡점',
 '봉원점',
 '와룡점',
 '주교점',
 '상일점',
 '충신점',
 '당주점',
 '북창점',
 '시흥점',
 '방화점',
 '방산점',
 '신공덕점',
 '서린점',
 '강일점',
 '공릉점',
 '월계점',
 '오장점',
 '응봉점',
 '동대문점',
 '내발산점',
 '정릉점

In [ ]:
def replace_region_names(text):
    for name1 in region_name:
        if name1 in text:
            # 지점 이름이 텍스트에 포함되어 있다면 대체
            pattern = re.compile(re.escape(name1), re.IGNORECASE)
            text = pattern.sub("<region>", text)
        # else :
        #   for i in range(len(name1), 1, -1):
        #       part = name1[:i]  # 지점 이름에서 처음부터 i 글자까지의 부분
        #       if part in text:
        #           # 지점 이름의 일부가 텍스트에 포함되어 있다면 대체
        #           pattern = re.compile(re.escape(part), re.IGNORECASE)
        #           text = pattern.sub("<region>", text)
    return text


In [ ]:
replace_region_names('오랜만에 헬리오시티점에서 맛있게 식사했어요')

'오랜만에 <region>에서 맛있게 식사했어요'

In [ ]:
replace_region_names('오랜만에 헬리오에서 맛있게 식사했어요')

'오랜만에 헬리오에서 맛있게 식사했어요'

In [ ]:
for i, review in tqdm(enumerate(train['reply']), desc="Processing", ncols=100):
    train['reply'][i] = replace_region_names(review)

Processing: 0it [00:00, ?it/s]

In [ ]:
changed_rows = train[train['reply'].str.contains("<region>")]
changed_rows

,review,reply,store
380,치즈추가하니 정말많더라구요~ 그리고 핫도그 강추! 다음엔 핫도그 많이 시킬꺼에요,<region> 에서만 드실수 있는 핫도그는 아주 인기가 좋은 녀석입니다. 또 찾아...,
382,아이가 좋아해요.치즈도 다른데보다 맛있다고 하고. 다음엔 중간정도 매운거 도전해 볼게요,저는 보통 입맛인데 순한맛이 너무 순해서 처음에는 안팔고 있었답니다. 아이가 있으시...,떡깨비
384,완전 맛있어요! 계속 여기서만 먹는 것 같아요. 감사히 잘 먹었습니다.,진짜 계속 문정<region> 에서만 드셔 주세요,
388,배송 엄청 빨라요!,"다행이네요, 배달이 계속 늘어날수록 배송이 늦어지고 있어서 고민이 많습니다. 더욱 ...",
391,오뎅이 쫄깃쫄깃하니 고급스런 오뎅이네요. 맛있게 잘 먹었습니다,"어? 저희 <region>에서 특별히 준비한 오뎅인데 마으메 드셨나봐요, 저희 오뎅...",
...,...,...,...
16708,요기 파이 완전 맛있어요~! 음료도 맛있구 파이도 맛있구 바삭바삭하고 다들 드세요 ...,반갑습니다 고객님~! <store> 청주<region> 을 찾아주셔서 진심으로 감사...,꼬알라파이
16709,풀드포크랑 미트파이랑 애플파이 먹음 파이는 손바닥만함 제 손이 좀 커서 한손으로 2...,안녕하세요 <store> 청주<region> 입니다 고객님의 이쁜 사진리뷰 하나하나...,꼬알라파이
16710,맛있네요 1리터 조아요,반갑습니다 고객님~! <store> 청주<region> 을 찾아주셔서 진심으로 감사...,꼬알라파이
17603,우선 요청도 안 했는데 튀김 만두 주셔서 감사합니다. 요거 땜시 리뷰 쓰는 건 아니...,정말 만족하게 드셨다니 고맙습니다 저희 시민족발보쌈 감자탕 가게는 이벤트성이 아닌 ...,


In [ ]:
for i, review in tqdm(enumerate(train['review']), desc="Processing", ncols=100):
    train['review'][i] = replace_region_names(review)

Processing: 0it [00:00, ?it/s]

In [ ]:
changed_rows = train[train['review'].str.contains("<region>")]
changed_rows

,review,reply,store
277,전반적으로 단맛이 좀 강하네요. 그래도 맛은 있습니다. 오늘 오후시간대 캔류 외 라...,안녕하세요 우선 죄송한 말씀 드립니다 그날 오후에 매장 주변 일대 수도관 청소로 잠...,
401,<region>에 처음 시켜먹었던게 맛있어서 주문했더니 돈 은 나갔는데 다시 앱을 ...,요기요가 주문어플에 렉이 자주 있어서. 여러가지로 혼돈이 있어 정말 죄송하게 되었습...,
1271,맛있어요 <region>은 리뷰이벤트로 꼬마물<store>있었는디 여기도 그러면 좋...,소중한 리뷰 감사드립니다 리뷰이벤트로 꼬마 물<store>가 없으셔서 많이 아쉬우셨...,육회
1389,김천재 <region>이 <store>에서 연어 <store> 젤 맛있어요! 육사시...,맛있게 드셔주셔서 감사드립니다 늘 믿고 드실 수 있게 항상 신선한 재료로 정성을 듬...,
3662,<store>비빔밥은 여기서만 시킵니다 진짜 너무맛있어요 집근처 지점말고 <regi...,"안녕하세요! 잊지 않으시고 저희 매장에 발자취 남겨주셔서 감사합니다 언제나 양, 맛...",
3892,제가 냉면을 좋아해서 배달 시킬 때마다 매번 식당을 바꿔가면서 먹습니다. 홈에 갈비...,"고객님, 시간내어 리뷰작성해주셔서 정말 감사합니다 항상 고객님들 주문 하나하나에 정...",갈비민족
3970,<region> 믿고먹습니다 항상 감사합니다.,"고객님, 안녕하세요 시간내어 리뷰작성해주셔서 정말 감사합니다 항상 고객님들 주문 하...",갈비민족
4096,급하게먹고잤는데 와 급하게먹어도 크으 족발에다가 월30이상쓰는데 여기랑 <regio...,항상 웃음이 가득한 <store>을 찾아주셔서 감사합니다 언제나 내가 먹는 음식이라...,구구족
5674,진짜 맛있어요 냉면집인데 <store>도 <region>만큼 맛있어요 최고,고객님 안녕하세요! 소중한 리뷰 너무 감사드려요 더욱 더 만족스러운 식사를 하실수 ...,대한냉면
6535,기존까지는 먹었던 돼지곱창이 일반 음식점에서 먹은 것이라면 여기는 돼지곱창 <reg...,안녕하세요.<store>입니다. 코로나로인해 고객님도 어려운시기에 찾아주셔서 감사하...,삽교원조두리곱창


In [ ]:
region_mask = []

for review in train['reply']:
  if '<region>' in review:
      print(review)

<region> 에서만 드실수 있는 핫도그는 아주 인기가 좋은 녀석입니다. 또 찾아 주시고 맛있게 드셔 주셔서 감사합니다. 
저는 보통 입맛인데 순한맛이 너무 순해서 처음에는 안팔고 있었답니다. 아이가 있으시면 순한을 많이 찾아 주시는거 같아요, 저희가 너무 자극적인 맛을 추구하지 않기 때문에 중간맛도 충분히 맛있게 드실수 있으실거 같아요, 다음에도 <store> <region>과 함께해 주세요. 감사합니다 
진짜 계속 문정<region> 에서만 드셔 주세요 
다행이네요, 배달이 계속 늘어날수록 배송이 늦어지고 있어서 고민이 많습니다. 더욱 맛있고 신속한 배송이 될수 있도록 할게요 감사합니다. <region> 
어? 저희 <region>에서 특별히 준비한 오뎅인데 마으메 드셨나봐요, 저희 오뎅은 시중에서 쉽게 구할수도 없고 오뎅전문 판매처에서만 파는 최고라고 알려진 오뎅업체이구요, 당연 단가도 비싸서 분식집들이 쉽게 판매하기 힘든 제품이란점 은근 자랑한번 해보겠습니다. 다른 재료들도 그렇지만 특히 떡,오뎅,순대,치즈는 자부심 가지고 있으니 또 주문해주시면 뭔가 다른퀄리티 음미하면서 즐겨봐 주세요, 감사합니다~ 
<store> 문정<region>은 배민에서 1인용도 테스트중 입니다. 곧 요기요도 반영할수 있도록 할게요 감사합니다~ 
저는 음식장사 야박하면 망한다 생각하는 사람으로서 최<store> 넉넉하게 드리려고 합니다. 분식이란게 여러가지 음식을 부담없이 드시는 그 재미또한 상당하기 때문에 여러가지 사이드와 토핑이 지속적으로 나오고 있으니 또 찾아주셔도 문정<region>은 계속 발전하고 기다릴게용 감사합니다~ 
요기요가 주문어플에 렉이 자주 있어서. 여러가지로 혼돈이 있어 정말 죄송하게 되었습니다. 문자로 상황설명 드리긴 했는데.<region>에서 기분이 나쁘셨는데.저희가 또 실수를 했네요.죄송합니다 연락주시면 다시 해드릴게요.
안녕하세요! 당신은 지금 치킨이 땡긴다 <region> 입니다 헉, 칭찬 너무 감사합니다 항상 한결같은 양과 맛으로 보답하겠습니다

## 4) 메뉴명 처리
-
- 일반 메뉴 리스트 -> 리스트 내에 있는 단어인 경우 확인해서 menu 칼럼에 추가 or 삭제
- 특수 메뉴 리스트 -> ex. 저희 ~~커피, 저희 ~~찜 등

In [ ]:
food_list = pd.read_csv(f'{DATA_PATH}전국통합식품영양성분정보(음식)표준데이터.csv', encoding='cp949')

In [ ]:
food_list = set(food_list['대표식품명'].tolist())

In [ ]:
food_list = list(food_list)

In [ ]:
ygy_menu_list = ['도시락','카레','커피','국밥','마라탕','마라샹궈','우동','나가사키','연어','치킨','쌀국수','닭강정','김치찜','김치찌개',
                '족발','보쌈','육회','갈비','피자','찜닭','파스타','냉면','곱창','닭발','막창','반찬','곱도리탕','순대','떡볶이','부대찌개'
                '삼겹살','분식','라면','버거','덮밥','초밥','통닭','짜장','짬뽕','탕수육','카페','와플','누들','김치','온면','디저트','라떼','고기','덮밥',
             '새우','돼지','닭','찌개','밥',
                 '양파','숙주','고수'] # 재료명 제거

In [ ]:
menu_name = food_list + ygy_menu_list

In [ ]:
len(menu_name)

814

In [ ]:
new_food_list = []  # 새로운 음식을 담을 리스트

for food in food_list:
    if '(' in food:
        food1, food2 = food.split('(')
        new_food_list.append(food1.strip())  # 공백 제거 후 리스트에 추가
        new_food_list.append(food2.rstrip(')'))
    else:
        new_food_list.append(food)

food_list = new_food_list

In [ ]:
menu_name = set(menu_name)
menu_name = list(menu_name)

menu_name = [name for name in menu_name if all(char.isalpha() for char in name)] #알파벳으로만 (한글) 구성된 경우남김. 숫자, 특수문자 지움


In [ ]:
def replace_menu_names(text):
    # 메뉴 이름을 긴 것부터 짧은 것 순으로 정렬
    sorted_menu_names = sorted(menu_name, key=len, reverse=True)

    for name1 in sorted_menu_names:
        if name1 in text:
            # 메뉴 이름이 텍스트에 포함되어 있다면 대체
            pattern = re.compile(re.escape(name1), re.IGNORECASE)
            text = pattern.sub("<menu>", text)
    return text


In [ ]:
for i, review in tqdm(enumerate(train['reply']), desc="Processing", ncols=100):
    train['reply'][i] = replace_menu_names(review)

Processing: 0it [00:00, ?it/s]

In [ ]:
changed_rows = train[train['reply'].str.contains("<menu>")]
changed_rows

,review,reply,store
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...,
5,양많고 맛있었어요,저희 매장 찾아주신 것도 감사한데 리뷰까지.감동입니다 고객님들의 배고픔을 저희는 지...,
16,저저번달에 맛있게 먹어서 또 시켰는데 고기 양이 엄청 줄었네요 .된장찌개도 엄청 짜...,저희 <store>의 메뉴를 주문해 주셔서 감사합니다. 양이 줄었다고 느끼셨군요. ...,1988응답하라추억의옛날도시락
23,20분 이내 배달 알림 주시고는 한시간이 지나서야 받았습니다. 배달이 늦는 점에 <...,안녕하세요 <store> 입니다. 남겨 주신 리뷰 잘 읽어보았습니다. 먼저 매장 상...,1988응답하라추억의옛날도시락
29,양도 많고 진짜 맛있어요! 시원한 묵사발로 더위도 가라앉히고 밥이랑 고기 양도 많아...,"저희 매장을 선택해주시고, 리뷰 남겨주셔서 감사합니다! 메뉴를 드시다가 양이 모자<...",
...,...,...,...
18615,늘 잘 먹고 있습니다~,"신메뉴 미니 <menu> 시리즈, 매운<menu><menu> 출시 새해 복 많이 ...",
18616,영쉬 믿고 먹는 <store> 덮밥입니당! 요청 드린대로 숙주랑 양파 더 주셔서 감...,"신메뉴 매운 <menu> <menu> , 미니 <menu> <menu> 출시 맛있...",
18620,연어초밥행사너무좋아요,<menu> <menu>입니다 헤헷 맛있게 드셔주셔서 감사합니당 리뷰 감사합니다! ...,
18636,스테이크 굽기 좋아서 너무 맛있게 먹었습니다. 고기 추가가 없어 아쉬웠지만. 덕분에...,고객님 요청으로 드디어 요기요에도 <menu>추가 추가 했습니다! 배.민에 이어 요...,


In [ ]:
for menu in menu_name:
    for i, review in enumerate(train['review']):
        if menu in review:
            train['review'][i] = replace_menu_names(review)

In [ ]:
changed_rows = train[train['review'].str.contains("<menu>")]
changed_rows

,review,reply,store
0,사진보다 실물이 백배 맛있어요~ <menu> 양도 많아요~ 배달시간도 딱 맞춰서 보...,"이렇게 저희 매장 찾아주셔서 감사드립니다! 맛 뿐만 아니라 양도, 서비스도 모두 최...",
9,새벽부터 시켰는데 배송 시간 맞춰서 잘 와주셨고 <menu>도 맛있고 묵사발도 맛있...,귀한 발걸음 감사합니다! 고객님의 리뷰는 저희를 성장시키는 원동력이 됩니다~ 내 가...,
16,저저<menu>달에 맛있게 먹어서 또 시켰는데 <menu> 양이 엄청 줄었네요 .<...,저희 <store>의 메뉴를 주문해 주셔서 감사합니다. 양이 줄었다고 느끼셨군요. ...,1988응답하라추억의옛날도시락
17,진짜 너무 자주 시켜먹어욧 소스가 삸싹 시큼한게 넘나 맛있고 <menu>도두툼하고 ...,안녕하세요! 저희 매장을 이용해 주셔서 진심으로 감사드려요! 저희의 메뉴가 만족스러...,
18,<menu>가 좀 매워요 맛있게 잘먹었습니다,"저희 매장을 선택해주시고, 리뷰 남겨주셔서 감사합니다 단순히 메뉴가 아니라 고객님의...",
...,...,...,...
18656,<menu> 매운소스 따로 부탁드렸는데 요청대로 해주셔서 맛있게 잘 먹었습니다~큐브...,맛있게 드셔주셔서 감사합니당 리뷰 감사합니다! 다음에 또 뵙겠습니다,
18658,<menu>도 넉넉히주시고 친절하세요 맛있습니다,맛있게 드셔주셔서 감사합니당 리뷰 감사합니다! 다음에 또 뵙겠습니다,
18659,맛있어요 <menu>도넉넉히주셨어요,맛있게 드셔주셔서 감사합니당 리뷰 감사합니다! 다음에 또 뵙겠습니다,
18679,맛있어요 <menu> 양 짱 많아요,감사합니다! 다음에도 이용해주세용,


In [ ]:
train.replace(['<menu><menu>','<menu> <menu>'], '<menu>')

,review,reply,store
0,사진보다 실물이 백배 맛있어요~ <menu> 양도 많아요~ 배달시간도 딱 맞춰서 보...,"이렇게 저희 매장 찾아주셔서 감사드립니다! 맛 뿐만 아니라 양도, 서비스도 모두 최...",
1,리뷰가 늦었네요. 맛있게 잘 먹었습니다. 또 시킬게요!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요 고객님께서 맛있다고 말씀 주시니 피로...,
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...,
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다! 저희 매장의 메뉴를 제대로 즐기신 거...,
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다 메뉴 그 자체로 행복을 드리고 싶었는데...,
...,...,...,...
18681,맛있게 잘 먹었어요,감사합니다! 다음에도 이용해주세용,
18682,요청사항을 항상 잘 들어주셨는데 <store> 못 모셨나봐요 그래도 잘 먹었습니다,죄송합니다 ! 다음에도 애용해주세용,
18683,배달 빨라요 맛있게 잘 먹었습니당 !,감사합니다! 다음에도 이용해주세용,
18684,잘 먹었어요~,감사합니다! 다음에도 이용해주세용,


## 5) 시간, 장소, 매장번호, 슬로건 등 고유 정보 처리
- 48시간, 24시간,02, 031, 번호 등

In [ ]:
# for reply in train['reply'] :


In [ ]:
train.to_csv(f'{DATA_PATH}train_token_masked.csv',index=False)

## 맞춤법

In [ ]:
!git clone https://github.com/ssut/py-hanspell.git

In [ ]:
!cd py-hanspell

In [ ]:
!pip install py-hanspell/

In [ ]:
from hanspell import spell_checker

def correct_spell(text):
    if isinstance(text, str):  # 입력 데이터가 문자열인 경우에만 맞춤법 검사 수행
        corrected_text = spell_checker.check(text).checked
        return corrected_text
    else:
        return text

In [ ]:
correct_spell(train['review'][0])

In [ ]:
train.head()

In [ ]:
correct_spell('감사합니다!!! 다음에도 이용해주세용')

In [ ]:
train['review'][:10].apply(correct_spell)

In [ ]:
train['reply'][:10].apply(correct_spell)

In [ ]:
# tqdm.pandas().  # 오래걸려서 우선 모델 학습시에 맞춤법 처리 된걸 넣을 필요는 없을 것 같고, 답변을 내뱉을때 맞춤법 처리 기능을 함수에 추가하기로

# train['review'] = train['review'].progress_apply(correct_spell)
# train['reply'] = train['reply'].progress_apply(correct_spell)

In [ ]:
# train.to_csv(f'{DATA_PATH}train_posneg_grammarchecked.csv',index=False)

# 데이터 분석/증강/변형

## 별점 비율 확인

In [ ]:
# train = pd.read_csv(f"{DATA_PATH}train_preprocessed.csv")

In [ ]:
train_rate = pd.concat([train, yogiyo['별점']],axis=1)
train_rate

In [ ]:
for i in range(1, 6):
    num_rating = train_rate[train_rate['별점'] == i].shape[0]
    globals()[f'rate_{i}'] = train_rate[train_rate['별점'] == i]
    print(f"별점 {i}: {num_rating}, {round(num_rating / train_rate.shape[0] * 100)}%")

In [ ]:
92+64+179

In [ ]:
rate_1

## 긍부정 비율 확인
- 식당 별점 평가 사전학습 모델 : MLOpsEngineer/review-rate-prediction
- Label_0 = 1점, Label_1 = 2점, Label_2 = 3점, Label_3 = 4점, Label_4 = 5점


In [ ]:
from transformers import pipeline

# 모델 불러오기
model_name = "MLOpsEngineer/review-rate-prediction"
classifier = pipeline("sentiment-analysis", model=model_name)


In [ ]:
classifier = pipeline("sentiment-analysis", model=model_name)

In [ ]:
print(train['review'][0])
classifier(train['review'][0])

In [ ]:
# 분류할 리뷰 텍스트
review_text = "맛있어요!"

# 모델 추론
result = classifier(review_text)[0]

# 결과 해석
label = result['label']
score = result['score']

if label in ['LABEL_0','LABEL_1'] :   # 어떤 기준으로 label 1, 2, 3을 잡을지 확인 필요. 대략은 0,1만 부정으로 하는게 좋아보임
    sentiment = '부정'
else:
    sentiment = '긍정'

# 출력
print(f"리뷰의 감정: {sentiment}")
print(f"긍부정 지수: {score}")


In [ ]:
import pandas as pd
from transformers import pipeline

def analyze_reviews(reviews):
    # 모델 불러오기
    model_name = "MLOpsEngineer/review-rate-prediction"
    classifier = pipeline("sentiment-analysis", model=model_name)

    results = []
    for review_text in tqdm(reviews):
        # 모델 추론
        result = classifier(review_text)[0]

        # 결과 해석
        label = result['label']
        score = result['score']

        if label in ['LABEL_0', 'LABEL_1']:
            sentiment = '부정'
        else:
            sentiment = '긍정'

        results.append((review_text, sentiment, score))

    # 데이터프레임 생성
    df = pd.DataFrame(results, columns=['review', 'sentiment', 'score'])
    return df

In [ ]:
# train_posneg = analyze_reviews(train['review'])

In [ ]:
# train_posneg

In [ ]:
# train_posneg.to_csv(f"{DATA_PATH}train_posneg.csv",index=False)
train_posneg = pd.read_csv(f"{DATA_PATH}train_posneg.csv")

In [ ]:
train_posneg

In [ ]:
cols = ['review','Unnamed: 0']
train_posneg = train_posneg.drop(columns=cols)
train_posneg

In [ ]:
train = pd.concat([train,train_posneg],axis=1,ignore_index=True)


In [ ]:
# train.to_csv(f"{DATA_PATH}train_posneg_done.csv",index=False)

In [ ]:
train.columns = ['review','reply','sentiment','score']

In [ ]:
# train_posneg[mask].shape[0]   # 893개 : 부정 : 별점 비율과 유사함  (부정 별점 1~4 : 895개, 1~3 : 335개)

## 데이터 증강 (부정 리뷰)

In [ ]:
train.head()

In [ ]:
train= pd.read_csv(f"{DATA_PATH}train_posneg_done.csv")

In [ ]:
train['store'].isnull().sum()

In [ ]:
mask_neg = train['sentiment'] == '부정'
train[mask_neg].shape[0]

In [ ]:
mask_pos = train['sentiment'] == '긍정'
train[mask_pos].shape[0]

## 1) resample 방법

In [ ]:
from sklearn.utils import resample

desired_samples = train[mask_pos].shape[0] - train[mask_neg].shape[0] #pos, neg개수 동일하게 하기 위해서
minority_class = train[train['sentiment'] == '부정']


In [ ]:
oversampled_data = resample(minority_class, replace=True, n_samples=desired_samples, random_state=42)
oversampled_data

In [ ]:
aug_train = pd.concat([train,oversampled_data],axis=0)
aug_train.shape

In [ ]:
aug_train.to_csv(f"{DATA_PATH}yogiyo_aug_train_token_masked.csv")

## 2) EDA (easy data augmentation)

In [ ]:
!pip install nlpaug

In [ ]:
# from nlpaug.util.text import random_word

# # EDA를 사용한 데이터 증강 예시 코드
# # 기존 리뷰를 다양한 방법으로 변형하여 새로운 리뷰 생성

# def eda_augmentation(review, alpha=0.1, num_aug=3):
#     augmented_reviews = []
#     words = review.split()

#     for _ in range(num_aug):
#         new_words = words.copy()
#         num_words_to_replace = max(1, int(alpha * len(words)))

#         for _ in range(num_words_to_replace):
#             rand_idx = random.randint(0, len(words) - 1)
#             new_words[rand_idx] = random_word(new_words[rand_idx])

#         augmented_reviews.append(' '.join(new_words))

#     return augmented_reviews

# # 부정 리뷰 데이터에 EDA 증강 적용
# final_oversampled_negative_reviews_eda = []
# for review in negative_reviews:
#     augmented_reviews = eda_augmentation(review)
#     final_oversampled_negative_reviews_eda.extend(augmented_reviews)


In [ ]:
# from imblearn.over_sampling import SMOTE
# from collections import Counter
# import pandas as pd

# # 소수 클래스와 다수 클래스 데이터 분리
# minority_class = train[train['sentiment'] == '부정']
# majority_class = train[train['sentiment'] == '긍정']

# # SMOTE 적용
# smote = SMOTE(sampling_strategy='auto', random_state=42)
# oversampled_minority = smote.fit_resample(minority_class[['review','reply']], minority_class['sentiment'])

# # 부정 리뷰 데이터 증강된 데이터셋 생성
# augmented_data = pd.concat([majority_class, pd.DataFrame(oversampled_minority, columns=['review','reply', 'sentiment'])])

# # 증강 결과 확인
# print(Counter(augmented_data['sentiment']))


## 3) google 스토어리뷰 활용

In [ ]:
google = pd.read_csv(f"{DATA_PATH}playstore_df.csv")
google.head(10)

# 데이터 학습

## 사전학습모델

In [ ]:
# aug_train.to_csv(f"{DATA_PATH}yogiyo_aug_train_token_masked.csv")

In [ ]:
shuffle_aug_train = aug_train.sample(frac=1, random_state=42)
train_ft = shuffle_aug_train[:3000].astype(str)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "skt/kogpt2-base-v2"
# model_name = "EasthShin/Youth_Chatbot_Kogpt2-base"

model = AutoModelForCausalLM.from_pretrained(model_name)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          bos_token='</s>',
                                          eos_token='</s>',
                                          unk_token='<unk>',
                                          pad_token='<pad>',
                                          mask_token='<mask>',
                                          max_len=1024)

tokenizer.eos_token_id # eos 토큰의 idx 반환

In [ ]:
tokenizer.tokenize(train['review'][0])

## 데이터셋 만들기

In [ ]:
train_ft = train_ft[['review','reply']]

In [ ]:
# class ChatDataset(torch.utils.data.Dataset):
#     def __init__(self,df):
#         self.review = df["review"].tolist()
#         self.reply = df["reply"].tolist()
#         # self.store = df["store"].tolist()

#     def __len__(self):
#         return len(self.review)

#     def __getitem__(self,idx):
#         return "<w>" + self.store[idx] + "</s><q>" + self.review[idx] + "</s><a>" + self.reply[idx] + "</s>"
#                 # 임의의 sos 토큰 <q>와 eos 토큰 </s> 삽입

In [ ]:
class ChatDataset(torch.utils.data.Dataset):
    def __init__(self,df):
        self.review = df["review"].tolist()
        self.reply = df["reply"].tolist()

    def __len__(self):
        return len(self.review)

    def __getitem__(self,idx):
        return "<w>" + "</s><q>" + self.review[idx] + "</s><a>" + self.reply[idx] + "</s>"

In [ ]:
def collate_fn(batch):
    x = tokenizer(batch, return_tensors="pt", padding=True)
    return {"x":x}

In [ ]:
train_dt = ChatDataset(train_ft)

In [ ]:
train_dl = torch.utils.data.DataLoader(train_dt,batch_size=2,collate_fn=collate_fn)

In [ ]:
batch = next(iter(train_dl))
batch

In [ ]:
tokenizer.pad_token_id # 패딩토큰인 3번 빼고 계산해야겠군

## 학습 loop함수화

In [ ]:
def train_loop(dataloader,model,optimizer,loss_fn,device):
    epoch_loss = 0
    model.train()

    for batch in tqdm(dataloader):
        x = batch["x"].to(device)
        pred = model(**x).logits # batch, seq, 단어별 실수값
        n_class = pred.shape[-1] # 클래스 개수
        pred = pred[:,:-1] # eos 토큰 제외
        pred = pred.reshape(-1,n_class) # 2차원 형태로 변환

        tgt = x["input_ids"][:,1:]
        tgt = tgt.flatten() # 1차원 벡터 형태로 변환

        mask = tgt != 3 # 패딩토큰인 3번 빼고 계산해야겠군
        tgt = tgt[mask]
        pred = pred[mask]
        loss = loss_fn(pred,tgt)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    epoch_loss /= len(dataloader)
    return epoch_loss

## 학습하기

In [ ]:
batch_size = 2
epochs = 10
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
train_ft['review'] = train_ft['review'].astype(str)
train_ft['reply'] = train_ft['reply'].astype(str)
# train_ft['store'] = train_ft['store'].astype(str)

In [ ]:
is_holdout = False
reset_seeds(SEED)

model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=3e-5)

train_dt = ChatDataset(train_ft)
train_dl = torch.utils.data.DataLoader(train_dt,batch_size=batch_size,shuffle=True,collate_fn=collate_fn)

for i in range(epochs):
    train_loss = train_loop(train_dl, model, optimizer, loss_fn, device)
    print(train_loss)

    if is_holdout:
        break

## 모델 저장하기

In [ ]:
model.save_pretrained(f"{DATA_PATH}yogiyo_kogpt2_model0901") #0.3073174620668093

## 답변 생성

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# model_name = 'MLOpsEngineer/review-rate-prediction'

# model = AutoModelForCausalLM.from_pretrained(f"{DATA_PATH}ygy_cont3000_youth_ver1")
model = AutoModelForCausalLM.from_pretrained(f"{DATA_PATH}yogiyo_kogpt2_model0901")

In [ ]:
model_name = "skt/kogpt2-base-v2"

tokenizer = AutoTokenizer.from_pretrained(model_name, #해당 모델의 토크나이저
                                          bos_token='</s>',
                                          eos_token='</s>',
                                          unk_token='<unk>',
                                          pad_token='<pad>',
                                          mask_token='<mask>',
                                          max_len=1024,
                                          padding_side='left')

In [ ]:
def correct_spell(text):
    if isinstance(text, str):  # 입력 데이터가 문자열인 경우에만 맞춤법 검사 수행
        corrected_text = spell_checker.check(text).checked
        return corrected_text
    else:
        return text

In [ ]:
# @torch.no_grad()
# def chatbot(model,tokenizer,max_len,device):
#     model.eval()
#     while True:
#         text = input("소비자 > ").strip()
#         if text == "quit":
#             break
#         text = "<q>" + text + "</s><a>"
#         x = tokenizer.encode(text,return_tensors="pt").to(device) # batch, seq

#         result_ids = model.generate(x,
#                             max_length=max_len,
#                             repetition_penalty=2.0,
#                             use_cache=True,
#                             do_sample=True, # 확률적 샘플링 여부(False = greedy 방식)
#                             temperature=0.2, # 소프트맥스 온도(확률적 샘플링일 때만 작동)
#                             top_k=5) # 상위 확률 k개 기준으로 확률적 샘플링 하겠다(확률적 샘플링일 때만 작동)
#         q_len = len(text) + 1
#         text = tokenizer.decode(result_ids[0])

#         from hanspell import spell_checker

#         text = correct_spell(text)

#         # if len(text) > max_len:
#         #     text = text[:max_len]

#         print("사장님 > ",text[q_len:-4])

In [ ]:

# chatbot 함수 정의
def chatbot(model, tokenizer, max_len, review_list):
    responses = []  # 결과를 저장할 리스트 초기화

    for review in review_list:
        # 대화 시작 및 종료 토큰 추가
        text = "<q>" + review + "</s><a>"

        # 입력 텍스트를 인코딩
        x = tokenizer.encode(text, return_tensors="pt")
        # max_len = 10000, max_length=max_len

        # 모델로부터 응답 생성
        result_ids = model.generate(x,
                                    max_length=max_len,
                                    num_return_sequences=1,
                                    repetition_penalty=2.0,
                                    use_cache=True,
                                    do_sample=True, # 확률적 샘플링 여부(False = greedy 방식)
                                    temperature=0.2, # 소프트맥스 온도(확률적 샘플링일 때만 작동)
                                    top_k=5) # 상위 확률 k개 기준으로 확률적 샘플링 하겠다(확률적 샘플링일 때만 작동)




        # 응답을 텍스트로 디코딩하여 저장
        response = tokenizer.decode(result_ids[0], skip_special_tokens=True)

        response = response.split("<a>", 1)[1]
        response = correct_spell(response)
        responses.append(response)

    # 결과를 데이터프레임으로 변환
    result_df = pd.DataFrame({"Review": review_list, "Response": responses})
    return result_df


In [ ]:
review_list = [

    '맛이 있어요',
    '최고에요',
    '사장님이 친절하시네요',
    '맛은 있는데 배달이 식어서 왔어요ㅠㅠ',
    '맛이 없어요',
    '이 머리카락 들어있는거 뭔가요?',
    '배달이 너무 불친절해요',
    '최악이에요'

]

# chatbot 실행 및 결과 출력
result_df = chatbot(model, tokenizer, 1024, review_list)

pd.DataFrame(result_df)

In [ ]:
result_df.to_csv(f'{DATA_PATH}yogiyo_kogpt2_model0901_1.csv',index=False)

In [ ]:
review_list = [
    # 특정 메뉴명 포함 (맛)
    '이집 피자 최애 넘나 맛있어요',
    '치킨이 바삭바삭하니 맛있네요',
    '마라샹궈 넘넘 맛있고 양도 푸짐해용! 감사합니다!',
    '간만에 술한잔생각나서 연어 주문해서 먹는데 맛있네요^^ 양이 좀적어진거같은데 다음에는 추가로 시켜야 겠어요^^ 잘먹었습니당^^',
    '짜장면 배달이 너무 느리게 와서 음식이 다 불었네요..사장님 신경좀 써주세요',
    '쌀국수 먹는데 비닐 나왔네요 반품 이런건 안하지만 청결에 신경 써주세요',
    '요기 김치찜 맛있는데 가끔 좀 짜요ㅠㅠ다음엔 육수 추가로 주심 좋겠어요',

    # 메뉴의 특성 포함 (맛)
    '닭도 쫄깃하니 맛있고 떡볶이도 맛나요👍',
    '오늘은 비계가 많은거 같아요.. 양이 상대적으로 적어서 아쉬웠어요.',
    '크림이 너무 부드럽고 맛있어요, 또 시켜먹을게요!!',
    '맛있게 먹었어요 4단계로 해도 맵지는 않아요',


    # 배달 및 기타 코멘트
    '배달 빠르게 와서 맛있게 먹었습니다!',
    '완전 만족했어요 전문 블로거인데요 양쪼끔만 더 많았으면! 종종 시켜먹을게요',
    '맛은 정말 좋았지만, 편의점가려다 배달 도착한걸 발견 못해다면 다 상한 음식 먹을뻔했네요. 노크도 없고 문자도 안주시면 음식이 온지 어떻게 알으라는건지....',
    '.................',
    '벌레 나왔어요.. 다신 안시켜먹을 것 같아요',
]

result_df = chatbot(model, tokenizer, 128, review_list)

pd.DataFrame(result_df)

In [ ]:

chatbot(model,tokenizer,128,device) # quit

# 답변 평가

## PPL

In [ ]:
import torch
import torch.nn as nn

# # 가정: 언어 모델 클래스가 이미 정의되어 있다고 가정
# class LanguageModel(nn.Module):
#     # 모델 정의 ...

# 퍼플렉시티 계산 함수
def calculate_perplexity(model, dataloader, loss_function):
    total_loss = 0.0
    total_tokens = 0

    with torch.no_grad():
        for batch in dataloader:
            inputs, targets = batch
            outputs = model(inputs)
            loss = loss_function(outputs, targets)
            total_loss += loss.item()
            total_tokens += targets.numel()

    avg_loss = total_loss / total_tokens
    perplexity = 2 ** avg_loss  # 퍼플렉시티 계산

    return perplexity

# 사용 예시
# model = LanguageModel(...)  # 언어 모델 클래스 초기화
# dataloader = torch.utils.data.DataLoader(...)  # 데이터로더 초기화
# loss_function = nn.CrossEntropyLoss()  # 손실 함수 초기화
# perplexity = calculate_perplexity(model, dataloader, loss_function)
# print(f"Perplexity: {perplexity}")


## BLEU

## METEOR

## SSA